In [1]:
from collections import defaultdict
from pymongo import MongoClient
import re
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from datetime import datetime
from nltk.tokenize import word_tokenize
import gensim

E:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train_df=pd.read_json("data/train_original_tweets_top1000.json")

In [15]:
def get_average_word_length(words,english_punctuations):
    word_count=0
    word_total_length=0
    for word in words:
        if word not in english_punctuations:
            word_total_length+=len(word)
            word_count+=1
    return word_total_length/word_count

In [ ]:
def generate_common_features(train_df):
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    train_df["tokenized_words"]=train_df["text"].apply(lambda x:word_tokenize(x))
    train_df["hashtag_num"]=train_df["entities"].apply(lambda x:len(x["hashtags"]))
    train_df["link_num"]=train_df["entities"].apply(lambda x:len(x["urls"]))
    train_df["mention_num"]=train_df["entities"].apply(lambda x:len(x["user_mentions"]))
    train_df["time_of_date"]=train_df["created_at"].apply(lambda x:x.hour*24+x.minute)
    train_df["len_of_tweet"]=train_df["text"].apply(lambda x:len(x))
    train_df["avg_len_of_word"]=train_df["tokenized_words"].apply(lambda x:get_average_word_length(x,english_punctuations))


In [ ]:
train_df["over_median_percentage"]=(train_df["retweet_count"]+1)/(train_df["retweet_median"]+1)
good_tweet_df=train_df[train_df["over_median_percentage"]>=10]
len(good_tweet_df)